# funbook

In [2]:
import numpy as np
import scipy as sc

import torch as tr
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import requests as req
import os
import ast

### decorators

In [3]:
import timeit
import random
from functools import wraps

def loggg(func_in=None, *, show_name=True, show_time=True):
    def example(f):
        @wraps(f)
        def func(*args, **kwargs):
            start = timeit.timeit()
            result = f(*args, **kwargs)
            end = timeit.timeit()
            print(end - start)
            print(f.__name__)
            print(result)
            return result
        return func

    # This is where the "magic" happens.
    if func_in is None:
        return example
    else:
        return example(func_in)

### board

In [4]:
with open('large_variables/moves.txt', 'r') as f:
            dic = f.read()
        
moves = ast.literal_eval(dic)

In [5]:
key = ((8, 0), (7, 1), (6, 2))
move = moves[key][1]
board = np.array([
                [2, 2, 2, 2, 2, 0, 0, 0, 0],
                [2, 2, 2, 2, 2, 2, 0, 0, 0],
                [1, 1, 2, 2, 2, 1, 1, 0, 0],
                [1, 1, 1, 1, 1, 1, 1, 1, 0],
                [1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 0],
                [1, 1, 3, 3, 3, 1, 1, 0, 0],
                [3, 3, 3, 3, 3, 3, 0, 0, 0],
                [3, 3, 3, 3, 3, 0, 0, 0, 0],
            ],dtype='i1',)
moves_black = {}

In [6]:
def r(entry):
    return np.array((entry[0],entry[1]+1),dtype='i1')

def l(entry):
        res = (entry[0],entry[1]-1)
        return np.array(res,dtype='i1')

def ru(entry):
    if(entry[0]<5):
        res = (entry[0]-1,entry[1])
    else:
        res = (entry[0]-1,entry[1]+1)
    return np.array(res,dtype='i1')

def lu(entry):
    if(entry[0]<5):
        res = (entry[0]-1,entry[1]-1)
    else:
        res = (entry[0]-1,entry[1])
    return np.array(res,dtype='i1')

def rd(entry):
    if(entry[0]<4):
        res = (entry[0]+1,entry[1]+1)
    else:
        res = (entry[0]+1,entry[1])
    return np.array(res,dtype='i1')

def ld(entry):
    if(entry[0]<4):
        res = (entry[0]+1,entry[1])
    else:
        res = (entry[0]+1,entry[1]-1)
    return np.array(res,dtype='i1')

def rt(t):
    if len(t) == 1:
        return np.array((r(t[0])))
    elif len(t) == 2:
        return np.array((r(t[0]),r(t[1])))
    else:
        return np.array((r(t[0]),r(t[1]),r(t[2])))

def lt(t):
    if len(t) == 1:
        return np.array((l(t[0])))
    elif len(t) == 2:
        return np.array((l(t[0]),l(t[1])))
    else:
        return np.array((l(t[0]),l(t[1]),l(t[2])))

def rut(t):
    if len(t) == 1:
        return np.array((ru(t[0])))
    elif len(t) == 2:
        return np.array((ru(t[0]),ru(t[1])))
    else:
        return np.array((ru(t[0]),ru(t[1]),ru(t[2])))

def lut(t):
    if len(t) == 1:
        return np.array((lu(t[0])))
    elif len(t) == 2:
        return np.array((lu(t[0]),lu(t[1])))
    else:
        return np.array((lu(t[0]),lu(t[1]),lu(t[2])))

def rdt(t):
    if len(t) == 1:
        return np.array((rd(t[0])))
    elif len(t) == 2:
        return np.array((rd(t[0]),rd(t[1])))
    else:
        return np.array((rd(t[0]),rd(t[1]),rd(t[2])))

def ldt(t):
    if len(t) == 1:
        return np.array((ld(t[0])))
    elif len(t) == 2:
        return np.array((ld(t[0]),ld(t[1])))
    else:
        return np.array((ld(t[0]),ld(t[1]),ld(t[2])))

def get_dir(key,move):
    if np.array_equal(rt(key),move):
        return 'r'
    elif np.array_equal(lt(key),move):
        return 'l'
    elif np.array_equal(rut(key),move):
        return 'ru'
    elif np.array_equal(lut(key),move):
        return 'lu'
    elif np.array_equal(rdt(key),move):
        return 'rd'
    elif np.array_equal(ldt(key),move):
        return 'ld'
    else:
        return ''

def advance(cur_pos,dir):
    """
    param current position
    param direction of movement
    return the current position advenced by the direction of movement
    """
    res = ''
    if dir == 'r':
        res = r(cur_pos)
    elif dir == 'l':
        res = l(cur_pos)
    elif dir == 'ru':
        res = ru(cur_pos)
    elif dir == 'lu':
        res = lu(cur_pos)
    elif dir == 'rd':
        res = rd(cur_pos)
    elif dir == 'ld':
        res = ld(cur_pos)
    else:
        raise Exception("IllegalDirection")
    
    if not((res[0] <= 8 and res[0] >= 0) and (res[1] <= 8 and res[1] >= 0)):
        raise Exception("IndexOutOfBoard")

    return res


In [7]:
def move_type(key,move):
        """
        param key
        param move
        return if the move has sideways
        """
        for i in key:
            for j in move:
                if np.array_equal(i,j):
                    return False
        return True

def opposite(color):
        """
        param color
        return the opposite color from the one which it is his turn
        """
        if color == 2:
            return 3
        else:
            return 2

def in_key(key,pos):
        """
        param key
        param position on the board
        return if the position is in the key
        """
        for i in key:
            if np.array_equal(i,pos):
                return True
        return False

In [8]:
in_key(key,[7,1])

True

In [9]:
def legal_move(key,move,color):
    if move_type(key,move):
        for pos in move:
            if board[pos[0]][pos[1]] != 1:
                return False
        return True
    
    else:
        dir = get_dir(key,move)
        cur_pos = np.copy(move[0])
        
        while in_key(key,cur_pos):
            try:
                cur_pos = advance(cur_pos,dir)
            except:
                break
        count = 0

        while board[cur_pos[0]][cur_pos[1]] == opposite(color):
            count += 1
            try:
                cur_pos = advance(cur_pos,dir)
            except:
                break

        return count < len(key) and board[cur_pos[0]][cur_pos[1]] != color


In [10]:
legal_move(key,move,3)

True

In [11]:
def on_board(key,color):
        """
        param positions on board
        param the color of the pieces
        return if the color is in the positions
        """
        for i in key:
            if board[i[0]][i[1]] != color:
                return False
        return True


In [12]:
def legal_moves(color):
    #passes over every key
    for key in moves:
        if on_board(key,color):

            to_add = list({})

            for move in moves[key]:
                if legal_move(key,move,color):
                    to_add.append(move)
            
            #checks where to add the moves
            if len(to_add) != 0:
                if color == 2:
                    moves_white[key] = np.array(to_add, dtype= 'i1')
                else:
                    moves_black[key] = np.array(to_add, dtype= 'i1')

In [13]:
def first(key,move):
    for i in key:
        if not in_key(move,i):
            return i

def move_(color,key,move):
    if move_type(key,move):
        for pos in key:
            board[pos[0]][pos[1]] = 0
        for pos in move:
            board[pos[0]][pos[1]] = color
    
    else:
        dir = get_dir(key,move)
        cur_pos = first(key,move)
        last_color = board[cur_pos[0]][cur_pos[1]]
        
        board[cur_pos[0]][cur_pos[1]] = 1
        
        while last_color != 1 and last_color != 0:
            try:
                cur_pos = advance(cur_pos,dir)
                cur_color = board[cur_pos[0]][cur_pos[1]]
                board[cur_pos[0]][cur_pos[1]] = last_color
                last_color = cur_color
            except:
                break


In [14]:
first(key,move)

(8, 0)

In [15]:
move_(3,key,move)

In [16]:
board

array([[2, 2, 2, 2, 2, 0, 0, 0, 0],
       [2, 2, 2, 2, 2, 2, 0, 0, 0],
       [1, 1, 2, 2, 2, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 3, 1, 1, 1, 1, 0],
       [1, 1, 3, 3, 3, 1, 1, 0, 0],
       [3, 3, 3, 3, 3, 3, 0, 0, 0],
       [1, 3, 3, 3, 3, 0, 0, 0, 0]], dtype=int8)

### intrface

### AI

### general